6. Object detection using Transfer Learning of CNN architectures
<br>
a. Load in a pre-trained CNN model trained on a large dataset
<br>
b. Freeze parameters (weights) in model’s lower convolutional layers
<br>
c. Add custom classifier with several layers of trainable parameters to model
<br>
d. Train classifier layers on training data available for task
<br>
e. Fine-tune hyper parameters and unfreeze more layers as needed

In [15]:
import os
os.listdir()


['.ipynb_checkpoints',
 '1 ir.docx',
 '1-comments in python.pdf',
 '10,12 pdf.pdf',
 '10,12 pdf_compressed (1).pdf',
 '10,12 pdf_compressed.pdf',
 '10th.pdf',
 '11148cnn_convolve_with_padding.gif',
 '113701963_PreviewExamForm (1).PDF',
 '113701963_PreviewExamForm.PDF',
 '12th.pdf',
 '19469imgg.webp',
 '19fc0dc2-18e4-429a-85a9-594afd691c61-mp4_720p.mp4',
 '1st activation fn.docx',
 '2 and 1.docx',
 '2. Features in Python.pdf',
 '2. String Functions-checkpoint (1).ipynb',
 '2. String Functions-checkpoint.ipynb',
 '2. Variables in Python.pdf',
 '2.py',
 '2012.05999v1.pdf',
 '20250910_0929_video.mp4',
 '2064c23d-8c72-4b43-8f71-6ee70be38a54-mp4_720p.mp4',
 '2304.13017v2.pdf',
 '2412.07750v1.pdf',
 '25_07_09_Linear_Regression_Gradient_Descent-20250821T174921Z-1-001.zip',
 '25_07_10_Linear_Regression-20250821T174925Z-1-001.zip',
 '25_07_11_Linear_Regression - Model-20250821T174930Z-1-001.zip',
 '25_07_14_Linear_Regression - Model-20250821T174934Z-1-001.zip',
 '2ir.docx',
 '3 image class.docx'

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

#### Pre processing img data 

In [17]:
dataset_dir = r"C:\Users\Jayesh Mahajan\Desktop\Dl\caltech-101-img"
dataset_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

# here batch_size is the number of images in each batch
batch_size = 2000
dataset_generator = dataset_datagen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 9144 images belonging to 102 classes.


#### Selecting first batch containing 2000 images as train and second batch containing 2000 images as test data

In [18]:
x_train, y_train =  dataset_generator[0]
x_test, y_test = dataset_generator[1]

print(len(x_train))
print(len(x_test))

2000
2000


#### a. Load in a pre-trained CNN model trained on a large dataset

In [23]:
# Load VGG16 without top layers
weights_path = r"C:\Users\Jayesh Mahajan\Desktop\Dl\vgg\vgg16_weights_tf_dim_ordering_tf_kernels_notop (1).h5"
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(64, 64, 3))
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


#### b. Freeze parameters (weights) in model’s lower convolutional layers

In [24]:
for layer in base_model.layers:
   layer.trainable = False

#### c. Add custom classifier with several layers of trainable parameters to model

In [25]:
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
predictions = Dense(102, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

#### d. Train classifier layers on training data available for task

In [26]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.1915 - loss: 3.9483 - val_accuracy: 0.2875 - val_loss: 3.3829
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.3585 - loss: 3.0139 - val_accuracy: 0.3960 - val_loss: 2.9026
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 111s 4s/step - accuracy: 0.4600 - loss: 2.4920 - val_accuracy: 0.4515 - val_loss: 2.5750
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 162s 4s/step - accuracy: 0.5310 - loss: 2.0948 - val_accuracy: 0.5020 - val_loss: 2.3311
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step - accuracy: 0.6195 - loss: 1.7728 - val_accuracy: 0.5100 - val_loss: 2.1894
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 127s 4s/step - accuracy: 0.6555 - loss: 1.5367 - val_accuracy: 0.5320 - val_loss: 2.0660
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.7185 - loss: 1.3443 - val_accuracy: 0.5445 - val_loss: 1.9680
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.7500 - loss: 1.1778 - val_accuracy: 0.5565 - val_

#### e. Fine-tune hyper parameters and unfreeze more layers as needed

In [27]:
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(64, 64, 3))
# freeze all layers first
for layer in base_model.layers:
   layer.trainable = False
# unfreeze last 4 layers of base model
for layer in base_model.layers[len(base_model.layers) - 2:]:
   layer.trainable = True
# fine-tuning hyper parameters
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = Dense(102, activation='sofstmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# training fine tuned model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 115s 4s/step - accuracy: 0.3515 - loss: 3.1718 - val_accuracy: 0.4785 - val_loss: 2.3841
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 132s 3s/step - accuracy: 0.5610 - loss: 1.8659 - val_accuracy: 0.5850 - val_loss: 1.8532
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.7010 - loss: 1.2021 - val_accuracy: 0.6015 - val_loss: 1.7402
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.7920 - loss: 0.7840 - val_accuracy: 0.6085 - val_loss: 1.6767
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 51s 782ms/step - accuracy: 0.8690 - loss: 0.4706 - val_accuracy: 0.6235 - val_loss: 1.6721
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 785ms/step - accuracy: 0.9215 - loss: 0.2748 - val_accuracy: 0.6495 - val_loss: 1.5921
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 776ms/step - accuracy: 0.9520 - loss: 0.1820 - val_accuracy: 0.6315 - val_loss: 1.7750
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 24s 770ms/step - accuracy: 0.9700 - loss: 0.1340 - val_accuracy: 0.65

In [ ]:
import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)

In [ ]:
labels = list(dataset_generator.class_indices.keys())

In [ ]:
n = 1000
plt.imshow(x_test[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])